# 1단계

In [ ]:
import pandas as pd

# -----------------------------
# 0) hit 파일 로드
# -----------------------------
hit = pd.read_parquet('hit_score.parquet')

print("hit shape:", hit.shape)
print("hit cols:", hit.columns.tolist())

# 혹시 모를 imdb_id 타입 통일
hit["imdb_id"] = hit["imdb_id"].astype(str).str.strip()

# -----------------------------
# 1) 드라마 메인 파일 로드
# -----------------------------
drama = pd.read_parquet("00_drama_main.parquet")
print("drama shape:", drama.shape)
print("drama cols:", drama.columns.tolist())

# imdb_id 없으면 여기서 바로 멈춰야 함
if "imdb_id" not in drama.columns:
    raise KeyError("00_drama_main.parquet 에 'imdb_id' 컬럼이 없어. 컬럼명을 확인해줘.")

drama["imdb_id"] = drama["imdb_id"].astype(str).str.strip()

# -----------------------------
# 2) overview(줄거리) 컬럼 자동 탐색 (있으면 그걸로 진행)
# -----------------------------
overview_candidates = ["overview", "plot", "description", "summary", "storyline"]
overview_col = next((c for c in overview_candidates if c in drama.columns), None)

if overview_col is None:
    raise KeyError(
        f"줄거리 컬럼을 못 찾았어. 후보={overview_candidates}\n"
        f"drama 컬럼들에서 줄거리 컬럼명이 뭔지 알려줘."
    )

print("✅ overview column:", overview_col)

# -----------------------------
# 3) hit 쪽 imdb_id 중복 체크 & 정리 (있으면 작품 단위로 1행 만들기)
#    - 기본: 같은 imdb_id가 여러 행이면 평균으로 집계
# -----------------------------
dup_cnt = hit.duplicated("imdb_id").sum()
print("hit duplicated imdb_id rows:", dup_cnt)

if dup_cnt > 0:
    hit_agg = (
        hit.groupby("imdb_id", as_index=False)[["rating", "num_votes_log", "sentiment_score", "hit_score"]]
           .mean()
    )
else:
    hit_agg = hit.copy()

print("hit_agg shape:", hit_agg.shape)

# -----------------------------
# 4) merge (드라마 기준으로 hit_score 붙이기)
# -----------------------------
master = drama.merge(hit_agg[["imdb_id", "hit_score"]], on="imdb_id", how="left")

print("master shape:", master.shape)
print("hit_score null rate:", master["hit_score"].isna().mean())

# 분석용으로 최소 컬럼만 따로 만든 버전도 하나 만들어두면 편함
master_min = master[["imdb_id", overview_col, "hit_score"]].copy()
master_min = master_min.rename(columns={overview_col: "overview"})

print("master_min shape:", master_min.shape)
print(master_min.head())

# 5) 저장

# master.to_parquet("01_drama_hit_master_full.parquet", index=False)
master_min.to_parquet("01_drama_hit_master_min.parquet", index=False)

print("✅ saved:")
print(" - 01_drama_hit_master_min.parquet")

# 2단계

In [ ]:
import pandas as pd

# -----------------------------
# 0) 마스터 로드
# -----------------------------
PATH_MASTER = "01_drama_hit_master_min.parquet"   # 저장한 마스터 파일명에 맞춰 수정 가능
df = pd.read_parquet(PATH_MASTER)

print("loaded:", df.shape)
print("cols:", df.columns.tolist())

# -----------------------------
# 1) 필수 컬럼 체크
# -----------------------------
required = ["imdb_id", "overview", "hit_score"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"필수 컬럼이 없어: {missing} / 현재 컬럼: {df.columns.tolist()}")

# imdb_id 정리 (안전)
df["imdb_id"] = df["imdb_id"].astype(str).str.strip()

# -----------------------------
# 2) 길이 컬럼 생성
# -----------------------------
# overview가 NaN이면 길이를 0으로
df["overview"] = df["overview"].fillna("").astype(str)
df["len_overview"] = df["overview"].str.len()

print("\nlen_overview summary:")
print(df["len_overview"].describe(percentiles=[.01,.05,.10,.25,.50,.75,.90,.95,.99]))

# -----------------------------
# 3) 후보 컷(Threshold)별로 필터링 결과 비교
# -----------------------------
THRESHOLDS = [100, 150, 200]

def summarize_for_threshold(data: pd.DataFrame, t: int) -> dict:
    # 2단계 필터링 규칙:
    # - overview 결측/빈문자열 제거 (len_overview >= t로 커버)
    # - hit_score 결측 제거
    filtered = data.loc[(data["len_overview"] >= t) & (data["hit_score"].notna())].copy()

    return {
        "threshold": t,
        "rows_after": len(filtered),
        "unique_imdb_id": filtered["imdb_id"].nunique(),
        "overview_empty_rate_before": (data["len_overview"].eq(0)).mean(),
        "hit_score_null_rate_before": data["hit_score"].isna().mean(),
        "hit_score_null_rate_after": filtered["hit_score"].isna().mean(),
        "len_overview_p50_after": filtered["len_overview"].median() if len(filtered) else None,
        "len_overview_p10_after": filtered["len_overview"].quantile(0.10) if len(filtered) else None,
    }

summary_rows = [summarize_for_threshold(df, t) for t in THRESHOLDS]
summary_df = pd.DataFrame(summary_rows).sort_values("threshold")

print("\n📌 Threshold comparison (Step2 filtering result)")
print(summary_df.to_string(index=False))

# -----------------------------
# 4) (선택) 기준 하나를 골라 2단계 산출물로 저장
# -----------------------------
# 일단 기본값은 150으로 설정 (나중에 검증 후 바꾸면 됨)
SELECTED_T = 150

df_step2 = df.loc[(df["len_overview"] >= SELECTED_T) & (df["hit_score"].notna()),
                  ["imdb_id", "overview", "hit_score", "len_overview"]].copy()

OUT_PATH = f"02_drama_filtered_len{SELECTED_T}.parquet"
df_step2.to_parquet(OUT_PATH, index=False)

print(f"\n✅ saved step2 file: {OUT_PATH}")
print("step2 shape:", df_step2.shape)
print(df_step2.head(3))

# df1 = pd.read_parquet('02_drama_filtered_len100.parquet')
df2 = pd.read_parquet('02_drama_filtered_len150.parquet')
# df3 = pd.read_parquet('02_drama_filtered_len200.parquet')

# 3단계

In [ ]:
import re
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# =========================================================
# ✅ 4단계 CONFIG (여기만 건드리면 됨)
# =========================================================
T = 150                 # overview 최소 길이
P = 0.20                # 상/하위 극단 비율 (hit/nonhit)

TOPN = 30               # 키워드 몇 개 뽑을지
MAX_FEATURES = 50000    # 단어 최대 개수 (많을수록 느림)
NGRAM_RANGE = (1, 2)    # (1,1) or (1,2) 추천
MIN_DF = 3              # 너무 희귀한 단어 제거 (2~5 추천)
MAX_DF = 0.80           # 너무 흔한 단어 제거 (0.7~0.9 추천)

# 토큰 규칙: 3글자 이상 영어만 (la, tv 같은 2글자 제거)
TOKEN_PATTERN = r"(?u)\b[a-z]{3,}\b"

# ✅ 커스텀 Stopwords (필요하면 계속 추가)
CUSTOM_STOPWORDS = {
    # 너무 일반적
    "set","new","help","team","one","two","three","also","may","like","makes","find",
    "based","story","series","show","episode","season","drama",

    # 서사에서 의미 약한 메타 단어
    "life","family","world","time","day","years","year","man","woman","girl","boy",

    # 제작/설명 계열
    "produced","production","television","tv",

    # 자주 튀는 짧은 토큰/로마숫자
    "iii","ii","iv","vi","vii","viii","ix","la",
}

# ✅ 추가로 지명/인명이 계속 튀면 여기에 계속 넣기
CUSTOM_STOPWORDS_EXTRA = {
    # 지명/국가/지역/인명(지금 튀는 것들)
    "york","america","england","mexico","colombia","john","jimmy",
    "city",  # york city 분해 방지용(원하면 빼도 됨)

    # 기관/메타
    "fbi","televisa","remake",

    # 너무 애매한 일반 단어(상황 따라)
    "epic","boss","century","force","history","abilities","goes","lost","money",
}

# 파일 경로
IN_PATH = f"02_drama_filtered_len{T}.parquet"

OUT_LABELED = f"04_labeled_len{T}_p{int(P*100)}.parquet"
OUT_HIT_TOP = f"04_keywords_hit_len{T}_p{int(P*100)}.csv"
OUT_NON_TOP = f"04_keywords_nonhit_len{T}_p{int(P*100)}.csv"
OUT_DELTA   = f"04_keywords_delta_len{T}_p{int(P*100)}.csv"

# =========================================================
# 1) 텍스트 클린 (가볍고 재현성 좋게)
# =========================================================
def clean_text(s: str) -> str:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return ""
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)   # 영어/숫자/공백만
    s = re.sub(r"\s+", " ", s).strip()
    return s


# =========================================================
# 2) 데이터 로드 + 품질 체크
# =========================================================
df = pd.read_parquet(IN_PATH)

need_cols = ["imdb_id", "overview", "hit_score"]
missing = [c for c in need_cols if c not in df.columns]
if missing:
    raise KeyError(f"필수 컬럼 누락: {missing} / 현재 컬럼: {df.columns.tolist()}")

keep_cols = need_cols + ([c for c in ["len_overview"] if c in df.columns])
df = df[keep_cols].copy()
df = df.dropna(subset=["imdb_id", "overview", "hit_score"]).copy()

df["overview_clean"] = df["overview"].map(clean_text)
df = df[df["overview_clean"].str.len() > 0].copy()

print("✅ loaded rows:", len(df))
print("✅ unique imdb_id:", df["imdb_id"].nunique())


# =========================================================
# 3) 라벨링 적용 (확정 기준 P)
# =========================================================
q_low = df["hit_score"].quantile(P)
q_high = df["hit_score"].quantile(1 - P)

nonhit = df[df["hit_score"] <= q_low].copy()
hit = df[df["hit_score"] >= q_high].copy()

nonhit["label"] = 0
hit["label"] = 1

labeled = pd.concat([nonhit, hit], ignore_index=True)

hit_n = int((labeled["label"] == 1).sum())
non_n = int((labeled["label"] == 0).sum())

print("\n📌 labeling report")
print(f"- T: {T} (min overview length)")
print(f"- P: {P}")
print(f"- q_low: {q_low:.6f}")
print(f"- q_high: {q_high:.6f}")
print(f"- hit_n: {hit_n}")
print(f"- nonhit_n: {non_n}")
print(f"- total_after: {len(labeled)}")

# =========================================================
# 4) TF-IDF 학습 (커스텀 가능한 버전)
# =========================================================
stop_words = set(ENGLISH_STOP_WORDS) | set(CUSTOM_STOPWORDS) | set(CUSTOM_STOPWORDS_EXTRA)

vec = TfidfVectorizer(
    stop_words=list(stop_words),
    max_features=MAX_FEATURES,
    ngram_range=NGRAM_RANGE,
    min_df=MIN_DF,
    max_df=MAX_DF,
    token_pattern=TOKEN_PATTERN,
)

X = vec.fit_transform(labeled["overview_clean"].tolist())
terms = np.array(vec.get_feature_names_out())

labels = labeled["label"].to_numpy()
hit_mask = labels == 1
non_mask = labels == 0

hit_mean = X[hit_mask].mean(axis=0).A1
non_mean = X[non_mask].mean(axis=0).A1
delta = hit_mean - non_mean

# =========================================================
# 5) Top 키워드 추출 + 저장
# =========================================================
def top_df(scores: np.ndarray, n: int = TOPN) -> pd.DataFrame:
    idx = np.argsort(scores)[::-1][:n]
    return pd.DataFrame({"keyword": terms[idx], "score": scores[idx]})

hit_top_df = top_df(hit_mean, TOPN).rename(columns={"score": "hit_mean_tfidf"})
non_top_df = top_df(non_mean, TOPN).rename(columns={"score": "nonhit_mean_tfidf"})

delta_pos_df = top_df(delta, TOPN).rename(columns={"score": "delta(hit-nonhit)"})
delta_neg_df = top_df(-delta, TOPN).rename(columns={"score": "delta(nonhit-hit)"})

delta_out = pd.DataFrame({
    "keyword": pd.concat([delta_pos_df["keyword"], delta_neg_df["keyword"]], ignore_index=True),
    "direction": (["hit+"] * TOPN) + (["nonhit+"] * TOPN),
    "delta_value": pd.concat([delta_pos_df["delta(hit-nonhit)"], delta_neg_df["delta(nonhit-hit)"]], ignore_index=True),
})

# =========================================================
# 6) 프리뷰 출력
# =========================================================
print("\n🏁 Preview (delta hit+ top 15)")
print(delta_pos_df.head(15).to_string(index=False))

print("\n🏁 Preview (delta nonhit+ top 15)")
print(delta_neg_df.head(15).to_string(index=False))

print("\n🧩 Debug tips")
print(f"- vocab size: {len(terms)}")
print(f"- NGRAM_RANGE: {NGRAM_RANGE}, MIN_DF: {MIN_DF}, MAX_DF: {MAX_DF}")
print(f"- stopwords size: {len(stop_words)}")


# =========================================================
# 7) 파일저장
# =========================================================

labeled.to_parquet(OUT_LABELED, index=False)
hit_top_df.to_csv(OUT_HIT_TOP, index=False)
non_top_df.to_csv(OUT_NON_TOP, index=False)
delta_out.to_csv(OUT_DELTA, index=False)

print("✅ saved labeled:", OUT_LABELED)
print("\n✅ saved outputs")
print("-", OUT_HIT_TOP)
print("-", OUT_NON_TOP)
print("-", OUT_DELTA)

In [1]:
import pandas as pd

In [2]:
df_delta = pd.read_csv('04_keywords_delta_len150_p20.csv')
df_hit = pd.read_csv('04_keywords_hit_len150_p20.csv')
df_non = pd.read_csv('04_keywords_nonhit_len150_p20.csv')
df = pd.read_parquet('04_labeled_len150_p20.parquet')

In [13]:
df.shape

(934, 6)

In [12]:
df_delta.shape

(60, 3)

In [15]:
df_hit.head(30)

,keyword,hit_mean_tfidf
0,lives,0.017419
1,young,0.013895
2,people,0.012907
3,police,0.012423
4,town,0.012106
5,murder,0.011275
6,high,0.011081
7,way,0.010748
8,crime,0.010346
9,detective,0.010235


In [16]:
df_non.head(30)

,keyword,nonhit_mean_tfidf
0,love,0.023442
1,young,0.017597
2,lives,0.014105
3,telenovela,0.013258
4,group,0.012688
5,school,0.012409
6,death,0.011935
7,daughter,0.011872
8,old,0.011614
9,murder,0.011508
